# Multi Location, Multiple Temporal Scales, Multiple Operations Including Transport, Mixed Integer Linear Programming Example

## [Example 4]

This is a continuation of 'One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming with Material and Emission Considerations Example' [Example 3]. 

We update Example 3 to now allow multiple locations and Transport options between them.


## From Example 3

In [1]:
from energia import *

m = Model('example4')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

In [2]:
m.periods

[q, y]

## Declaring Locations 

In [3]:
m.ho = Location(label='Houston')
m.sd = Location(label='San Diego')
m.ny = Location(label='New York')
# m.usa = m.ho + m.sd + m.ny

In [4]:
m.locations

[ho, sd, ny]

### The Network 

Given that we have not used a nested set of locations, a network is made:

In [5]:
m.network

ntw

This network contains the three locations at (ho, sd, ny)

In [6]:
m.network.has

(sd, ho, ny)

## General Bounds Default to Network

In [7]:
m.declare(Resource, ['power', 'wind', 'solar'])
# m.solar.consume == True
# m.wind.consume == True
m.solar.consume <= 1200
m.wind.consume <= 1200
# m.consume.show()

--- General Resource Balance for solar in (ntw, y): initializing constraint, adding consume(solar, ntw, y)
    Completed in 0.00013589859008789062 seconds
--- Binding consume in domain (solar, ntw, y)
    Completed in 5.817413330078125e-05 seconds
--- General Resource Balance for wind in (ntw, y): initializing constraint, adding consume(wind, ntw, y)
    Completed in 6.771087646484375e-05 seconds
--- Binding consume in domain (wind, ntw, y)
    Completed in 4.38690185546875e-05 seconds


## Location-specific Bounds 

Spatial specificity can be provided along with temporal fidelity. 
In the example below, the temporal fidelity is implied since the data has 4 samples.

Note that an individual general resource balance is generated for power at each location

In [8]:
m.power.release(m.ho).prep(30) >= [0.6, 0.7, 0.8, 0.3]
m.power.release(m.sd).prep(100) >= [0.4, 0.9, 0.7, 0.6]
m.power.release(m.ny).prep(180) >= [0.2, 0.5, 0.7, 0.5]
m.release.show()

--- General Resource Balance for power in (ho, q): initializing constraint, adding release(power, ho, q)
    Completed in 0.0001475811004638672 seconds
--- Binding release in domain (power, ho, q)
    Completed in 0.00017189979553222656 seconds
--- General Resource Balance for power in (sd, q): initializing constraint, adding release(power, sd, q)
    Completed in 9.942054748535156e-05 seconds
--- Binding release in domain (power, sd, q)
    Completed in 0.00014591217041015625 seconds
--- General Resource Balance for power in (ny, q): initializing constraint, adding release(power, ny, q)
    Completed in 0.0002541542053222656 seconds
--- Binding release in domain (power, ny, q)
    Completed in 0.00011944770812988281 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## For All Locations 

Generally, in Energia to repeat a constraint over a set, use .forall()

If a list is provided, the bounds are iterated over the list [see LB cons]
else the bounds are repeated [see UB cons]

In [9]:
m.wf = Process(m.power)
m.wf(m.power) == -1 * m.wind
[10, 7, 20] <= m.wf.capacity.x.forall(m.network.has)
m.wf.capacity.x.forall(m.network.has) <= 100
m.wf.show()

--- Binding capacity in domain (wf, sd, y)
    Completed in 0.00016069412231445312 seconds
--- Binding capacity in domain (wf, ho, y)
    Completed in 0.00010466575622558594 seconds
--- Binding capacity in domain (wf, ny, y)
    Completed in 9.560585021972656e-05 seconds
--- Binding capacity in domain (wf, sd, y)
    Completed in 0.0001010894775390625 seconds
--- Binding capacity in domain (wf, ho, y)
    Completed in 9.226799011230469e-05 seconds
--- Binding capacity in domain (wf, ny, y)
    Completed in 8.797645568847656e-05 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Bounds can also be provided individually

In [10]:
m.wf.operate.prep(norm=True)(m.ho) <= [0.9, 0.8, 0.5, 0.7]
m.wf.operate.prep(norm=True)(m.sd) <= [0.8, 0.7, 0.6, 0.5]
m.wf.operate.prep(norm=True)(m.ny) <= [0.6, 0.5, 0.4, 0.3]
m.operate.show(True)
# m.wf.operate.prep(norm = True).forall(m.network.has) <= [[0.9, 0.8, 0.5, 0.7], [0.8, 0.7, 0.6, 0.5], [0.6, 0.5, 0.4, 0.3]]

--- Binding operate in domain (wf, ho, q)
    Completed in 0.00021409988403320312 seconds
--- Binding operate in domain (wf, sd, q)
    Completed in 0.00030040740966796875 seconds
--- Binding operate in domain (wf, ny, q)
    Completed in 0.00017213821411132812 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Calculations can also be provided for all locations as a list or repeated

In [11]:
m.wf.capacity[m.usd.spend].forall(m.network.has) == [90000, 70000, 60000]
m.wf.operate[m.usd.spend] == 49

--- Mapping spend: (usd, ho, y, capacity, wf) → (usd, ntw, y, operate, wf)
    Completed in 0.000s
--- Mapping spend: (usd, sd, y, capacity, wf) → (usd, ntw, y, operate, wf)
    Completed in 0.000s
--- Mapping spend: (usd, ny, y, capacity, wf) → (usd, ntw, y, operate, wf)
    Completed in 0.000s


In the case where location specific data is not provided, the parameter is assumed tuo apply to the network

In [12]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar

In [13]:
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True).forall(m.network.has) <= [
    [0.6, 0.8, 0.9, 0.7],
    [0.5, 0.7, 0.6, 0.5],
    [0.4, 0.6, 0.5, 0.4],
]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

m.gwp = Environ()

m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000

m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17
m.cement.consume[m.gwp.emit] == 0.9

m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

--- Binding capacity in domain (pv, ntw, y)
    Completed in 0.000152587890625 seconds
--- Binding capacity in domain (pv, ntw, y)
    Completed in 0.00010704994201660156 seconds
--- Binding operate in domain (pv, sd, q)
--- Aspect (capacity) not defined at sd, a variable will be created assuming y as the temporal index
--- Mapping capacity: (pv, sd, y) → (pv, ntw, y)
    Completed in 0.000s
    Completed in 0.00035309791564941406 seconds
--- Binding operate in domain (pv, ho, q)
--- Aspect (capacity) not defined at ho, a variable will be created assuming y as the temporal index
--- Mapping capacity: (pv, ho, y) → (pv, ntw, y)
    Completed in 0.000s
    Completed in 0.00035691261291503906 seconds
--- Binding operate in domain (pv, ny, q)
--- Aspect (capacity) not defined at ny, a variable will be created assuming y as the temporal index
--- Mapping capacity: (pv, ny, y) → (pv, ntw, y)
    Completed in 0.000s
    Completed in 0.0005171298980712891 seconds
--- Mapping spend: (usd, ho, y

## Linkages 

A grid of valid linkages can be created from sources to sinks.

Linkages are always one directional, stating bi, created a link from source to sink and the other way round 

If multiple links exist between two locations with different distances, it is necessary to create named links. 
Again, it needs to be stated whether the links are bi directional 

In [14]:
m.Link(source=m.ho, sink=m.sd, dist=2000, bi=True)
# m.Link(source = m.sd, sink=m.ny, dist = 1500)
m.road = Linkage(source=m.ho, sink=m.ny, dist=3000, bi=True)
# m.rail = Link(source = m.ho, sink=m.ny, dist = 5000, bi = True)
m.linkages

[ho-sd, sd-ho, road, -road]

In [15]:
m.ho.links(m.sd)

ho is source and sd is sink in ho-sd
sd is source and ho is sink in sd-ho


[ho-sd, sd-ho]

## Transits

Declare some transport options between across the linkages 


In [16]:
m.declare(Resource, ['h2o', 'co2'])
m.grid = Transport()
m.grid(m.power) == -m.h2o + m.co2
m.grid.capacity.x.forall([m.ho - m.sd, m.road]) <= 100
m.grid.operate.prep(norm=True).forall([m.ho - m.sd, m.road]) <= [
    [0.9, 0.8, 0.5, 0.7],
    [0.8, 0.7, 0.6, 0.9],
]
m.grid.operate[m.usd.spend] == 2000
m.grid.capacity[m.usd.spend] == 1000

--- Binding capacity in domain (grid, ho-sd, y)
    Completed in 0.0001621246337890625 seconds
--- Binding capacity in domain (grid, road, y)
    Completed in 0.00010347366333007812 seconds
--- Binding operate in domain (grid, ho-sd, q)
    Completed in 0.00014781951904296875 seconds
--- Binding operate in domain (grid, road, q)
    Completed in 0.00012826919555664062 seconds
--- Mapping spend: (usd, ho, y, capacity, wf) → (usd, ntw, y, operate, grid)
    Completed in 0.000s
--- Mapping spend: (usd, sd, y, capacity, wf) → (usd, ntw, y, operate, grid)
    Completed in 0.000s
--- Mapping spend: (usd, ny, y, capacity, wf) → (usd, ntw, y, operate, grid)
    Completed in 0.000s
--- Mapping spend: (usd, ho, y, capacity, wf) → (usd, ntw, y, capacity, grid)
    Completed in 0.000s
--- Mapping spend: (usd, sd, y, capacity, wf) → (usd, ntw, y, capacity, grid)
    Completed in 0.000s
--- Mapping spend: (usd, ny, y, capacity, wf) → (usd, ntw, y, capacity, grid)
    Completed in 0.000s


In [17]:
m.ho.locate(m.pv, m.wf, m.lii)
m.sd.locate(m.pv, m.wf, m.lii)
m.ny.locate(m.pv, m.wf, m.lii)

--- Assuming  pv capacity is unbounded in (ho, y)
--- General Resource Balance for power in (ho, q): adding produce(power, ho, q, operate, pv)
    Completed in 0.00011277198791503906 seconds
--- General Resource Balance for solar in (ho, y): initializing constraint, adding expend(solar, ho, y, operate, pv)
    Completed in 6.580352783203125e-05 seconds
--- Mapping operate: (pv, ho, q) → (pv, ho, y)
    Completed in 0.000s
--- Mapping operate: (pv, ho, y) → (pv, ntw, y)
    Completed in 0.000s
--- General Resource Balance for power in (ho, q): adding produce(power, ho, q, operate, wf)
    Completed in 0.00010800361633300781 seconds
--- General Resource Balance for wind in (ho, y): initializing constraint, adding expend(wind, ho, y, operate, wf)
    Completed in 6.437301635742188e-05 seconds
--- Mapping operate: (wf, ho, q) → (wf, ho, y)
    Completed in 0.000s
--- Mapping operate: (wf, ho, y) → (wf, ntw, y)
    Completed in 0.000s
--- Assuming  power.lii inventory capacity is unbounded 

In [18]:
m.show()

# Mathematical Program for Program(example4)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [19]:
m.grb 

defaultdict(<function energia.represent.model.Model.__post_init__.<locals>.<lambda>()>,
            {solar: defaultdict(<function energia.represent.model.Model.__post_init__.<locals>.<lambda>.<locals>.<lambda>()>,
                         {ntw: defaultdict(list,
                                      {y: [consume(solar, ntw, y)], q: []}),
                          ho: defaultdict(list,
                                      {y: [expend(solar, ho, y, operate, pv)]}),
                          sd: defaultdict(list,
                                      {y: [expend(solar, sd, y, operate, pv)]}),
                          ny: defaultdict(list,
                                      {y: [expend(solar, ny, y, operate, pv)]})}),
             wind: defaultdict(<function energia.represent.model.Model.__post_init__.<locals>.<lambda>.<locals>.<lambda>()>,
                         {ntw: defaultdict(list,
                                      {y: [consume(wind, ntw, y)], q: []}),
                     

In [19]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [20]:
m.solar.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [21]:
m.usd.spend.opt()

--- Mapping spend: (usd, ho, y, capacity, wf) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, sd, y, capacity, wf) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, ny, y, capacity, wf) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, ntw, y, operate, wf) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, ntw, y, capacity, pv) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, ntw, y, operate, pv) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, ntw, y, invcapacity, power.lii) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, ntw, y, inventory, power.lii) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, ntw, y, consume, cement) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, ntw, y, operate, grid) → (usd, ntw, y)
    Completed in 0.000s
--- Mapping spend: (usd, ntw, y, capacity, grid) → (usd, ntw, y)
    Completed in 0.000s
--- Generating Program(e

In [22]:
m.sol()

'Use .opt() to generate solution'

In [23]:
m.export.sol()

AttributeError: example4 has no 'export'

In [ ]:
m.spend.sol()